In [1]:
from tkinter import *
import sklearn, numpy as np, scipy as scp, pandas as pd, matplotlib.pyplot as plt, matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.linear_model import LogisticRegression
import yfinance as yf

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer

margin = 0

In [2]:
root = Tk()
frame = Frame(root)
frame.pack()
bottomframe = Frame(root)

def setTesla():
    global comp
    comp = 'TSLA'
def setApple():
    global comp
    comp = 'AAPL'
def setBoeing():
    global comp
    comp = 'BA'

comp = ''
bottomframe.pack( side = BOTTOM )
redbutton = Button(frame, text = 'TSLA', fg = 'black', width = 25, command = setTesla)
redbutton.pack( side = LEFT)
greenbutton = Button(frame, text = 'AAPL', fg = 'black', width = 25, command = setApple)
greenbutton.pack( side = LEFT )
bluebutton = Button(frame, text = 'BA', fg = 'black', width = 25, command = setBoeing)
bluebutton.pack( side = LEFT )

var_start, var_end = DoubleVar(), DoubleVar()
label_historicalRange = Label(root, text = 'Historical Range: ')
label_historicalRange.pack()
label_historicalRange = Label(root, text = 'Start- ')
label_historicalRange.pack()
scale_start = Scale(root, from_=-365, to=0, orient=HORIZONTAL, variable = var_start)
scale_start.pack()
label_historicalRange = Label(root, text = 'End- ')
label_historicalRange.pack()
scale_end = Scale(root, from_=-365, to=0, orient=HORIZONTAL, variable = var_end)
scale_end.pack()

label_parameters = Label(root, text = 'Include: ').pack()
var1 = IntVar()
Checkbutton(root, text='Volume', variable=var1).pack()

def quit():
    root.quit()

button_quit = Button(root, text = 'Train', fg = 'black', command = quit).pack()

root.mainloop()

In [3]:
print(comp)
my_comp = yf.Ticker(comp)
data = my_comp.history(period='1y')
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data['close_yesterday'] = data['Close'].shift(1, fill_value=0)

data['volume_yesterday'] = data['Volume'].shift(1, fill_value = 0)

data['target_value'] = data.apply(lambda row: row.Close - row.Open, axis=1)
data['target_classifier'] = data.apply(lambda row: 1 if(row.target_value > margin) else 0, axis = 1)
data = data.shift(-1)
data.drop(data.tail(1).index,inplace=True)
y = data['target_classifier']
x = data.drop(['target_classifier','Close','Volume','target_value'],axis=1)
train_raw, test_raw, target, target_test = train_test_split(x,y, test_size=0.25, random_state=0)
pipeline = Pipeline([
    ('scaler', StandardScaler())
])

#Transform raw data 
train = pipeline.fit_transform(train_raw)
test = pipeline.transform(test_raw)
lr = LogisticRegression()
lr.fit(train,target)
predicted = lr.predict(test)
#predict_yest = lr.predict()
print(train[-1:])
yesterday = data[-1:]
yesterday = yesterday.drop(['target_classifier','Close','Volume','target_value'],axis=1)
yesterday = pipeline.fit_transform(yesterday)
predict_yest = lr.predict(yesterday)
print("%-12s %f" % ('Accuracy:', sklearn.metrics.accuracy_score(target_test,predicted)))
if predict_yest:
    print("We predict today would be a good day to buy")
else:
    print("Hold off on purchasing today")

BA
[[ 1.23606907  1.23977152  1.24820861  1.20653325 -0.54807603]]
Accuracy:    0.714286
We predict today would be a good day to buy
